In [1]:
!pip install numpy==1.26.4

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
!pip install ISLP


In [4]:
from ISLP import load_data

In [10]:


random_seed = np.random.seed(1515)

df = load_data("College")
df.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


(a) Split the data set into a training set and a test set.


In [13]:

X = df.drop("Apps", axis=1)
y = df["Apps"]


X = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

(b) Fit a linear model using least squares on the training set, and report the test error obtained.

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
y_pred_linreg = linreg.predict(X_test_scaled)
mse_linreg = mean_squared_error(y_test, y_pred_linreg)

print(mse_linreg)

642753.8976533783


(c) Fit a ridge regression model on the training set, with λ chosen by cross-validation. Report the test error obtained.

In [19]:
alphas = 10**np.linspace(10, -2, 100) * 0.5

ridge_cv = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error', store_cv_results=True)
ridge_cv.fit(X_train_scaled, y_train)


print(ridge_cv.alpha_)

0.005


In [21]:
y_pred_ridge = ridge_cv.predict(X_test_scaled)

mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print(mse_ridge)

642759.6350794964


(d) Fit a lasso model on the training set, with λ chosen by cross- validation. Report the test error obtained, along with the num- ber of non-zero coefficient estimates.



In [24]:
lasso_cv = LassoCV(alphas=alphas, cv=10, max_iter=100000)
lasso_cv.fit(X_train_scaled, y_train)

print(lasso_cv.alpha_)


lasso_y_pred = lasso_cv.predict(X_test_scaled)

lasso_mse = mean_squared_error(y_test, lasso_y_pred)
print(lasso_mse)


non_0_coef = sum(lasso_cv.coef_ != 0)
print(non_0_coef)

12.38538177995857
659831.8114908539
14


(e) Fit a PCR model on the training set, with M chosen by cross- validation. Report the test error obtained, along with the value of M selected by cross-validation.

In [35]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('linreg', LinearRegression())
])

param_grid = {'pca__n_components': list(range(1, X_train.shape[1] + 1))}

grid = GridSearchCV(pipe, param_grid, cv=cv, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

best_m = grid.best_params_['pca__n_components']
print(best_m)

y_pred_pcr = grid.predict(X_test)
mse_pcr = mean_squared_error(y_test, y_pred_pcr)
print(mse_pcr)

17
642753.8976533791


After running ridge, lasso, PCR, and a linear model and observing the test errors, we watched the errors range from ~642,000 to ~659,000. While there is some improvement, it is not a dramatic improvement, and the linear model produces one of the lowest test errors. The ridge regression's test error follows closely, suggesting the model does not benefit much from shrinkage. Lasso regression picked 14 non-zero coefficients, and had the worst test error at 659,831. The predictors dropped resulted in a higher error.